# Basic MNIST Classification Notebook

Notebook based on: https://github.com/RafayKhattak/Digit-Classification-Pytorch

## Versioning Check

In [4]:
import torch
import torchvision

assert torch.__version__ == "2.4.1"
assert torchvision.__version__ == "0.19.1"

## MNIST Classification

In [25]:
# Importing dependencies
import torch

from PIL import Image
from torch import nn,save,load
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
# Loading Data
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root="data", download=True, train=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:05<00:00, 1907979.60it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 56501.39it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:02<00:00, 670541.64it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2309434.93it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [4]:
# Define the image classifier model
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU()
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 22 * 22, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [29]:
# Create an instance of the image classifier model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier = ImageClassifier().to(device)

In [30]:
# Define the optimizer and loss function
optimizer = SGD(classifier.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [31]:
# Train the model
for epoch in range(5):  # Train for 5 epochs
    epoch_loss_sum, epoch_samples = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()  # Reset gradients
        outputs = classifier(images)  # Forward pass
        loss = loss_fn(outputs, labels)  # Compute loss
        epoch_loss_sum += loss.item() * len(images)
        epoch_samples += len(images)
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
    
    print(f"[Epoch {epoch}] Loss is {epoch_loss_sum/epoch_samples:.7f}")

[Epoch 0] Loss is 1.0994408
[Epoch 1] Loss is 0.3394061
[Epoch 2] Loss is 0.3158091
[Epoch 3] Loss is 0.3021494
[Epoch 4] Loss is 0.2915728


In [32]:
# Save the trained model
torch.save(classifier.state_dict(), 'model_state.pt')

In [33]:
# Load the saved model
with open('model_state.pt', 'rb') as f: 
     classifier.load_state_dict(load(f))  

/tmp/ipykernel_180105/408145653.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(load(f))


In [34]:
test_dataset = datasets.MNIST(root="data", download=True, train=False, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Test the model
correct, total = 0, 0
for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = classifier(images)
    predictions = outputs.argmax(dim=1)
    correct += (predictions == labels).sum().item()
    total += len(labels)

acc = correct / total
print(f"Accuracy on the test set: {acc*100:.2f}%")

Accuracy on the test set: 92.11%


In [ ]:
# Perform inference on an image
img_path = 'image.jpg'
img = Image.open(img_path)
img_transform = transforms.Compose([transforms.ToTensor()])
img_tensor = img_transform(img).unsqueeze(0).to(device)
output = classifier(img_tensor)
predicted_label = torch.argmax(output)
print(f"Predicted label: {predicted_label}")